<a href="https://colab.research.google.com/github/NeanderDanubio26827/BaseDeDadosTitanic/blob/main/AplicacaoNaivesBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código da implementação do Naive Bayes

In [ ]:
from optparse import Values
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer
import collections

In [ ]:
def faixa_etaria(idade):
    if idade == np.nan:
        return 'Não informada'
    elif '0' <= idade <= '10':
        return 'Criança'
    elif '11' <= idade <= '18':
        return 'Jovem'
    elif '19' <= idade <= '50':
        return 'Adulto'
    else:
        return 'Idoso'

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/NeanderDanubio26827/projetoKNN/main/CVLI_12_2021Tratado.csv' , sep=';')

df = df[['Natureza',  'Gênero', 'Idade da Vítima', 'Escolaridade da Vítima','Raça da Vítima']]
df.head()

,Natureza,Gênero,Idade da Vítima,Escolaridade da Vítima,Raça da Vítima
0,HOMICIDIO DOLOSO,Masculino,Não Informada,Ensino Fundamental Incompleto,Não Informada
1,LESAO CORPORAL SEGUIDA DE MORTE,Feminino,15,Ensino Fundamental Incompleto,Parda
2,HOMICIDIO DOLOSO,Masculino,16,Alfabetizado,Não Informada
3,ROUBO SEGUIDO DE MORTE (LATROCINIO),Masculino,20,Ensino Fundamental Completo,Não Informada
4,HOMICIDIO DOLOSO,Masculino,26,Alfabetizado,Não Informada


In [ ]:
df.isnull().sum()

Natureza                  0
Gênero                    0
Idade da Vítima           0
Escolaridade da Vítima    0
Raça da Vítima            0
dtype: int64

In [ ]:
df.isna().sum()

Natureza                  0
Gênero                    0
Idade da Vítima           0
Escolaridade da Vítima    0
Raça da Vítima            0
dtype: int64

In [ ]:
df['FaixaEtaria']= [faixa_etaria(k) for k in df['Idade da Vítima'].values]
df = df.drop('Idade da Vítima', axis=1)
print(df)

                                 Natureza     Gênero  \
0                        HOMICIDIO DOLOSO  Masculino   
1         LESAO CORPORAL SEGUIDA DE MORTE   Feminino   
2                        HOMICIDIO DOLOSO  Masculino   
3     ROUBO SEGUIDO DE MORTE (LATROCINIO)  Masculino   
4                        HOMICIDIO DOLOSO  Masculino   
...                                   ...        ...   
3294                     HOMICIDIO DOLOSO  Masculino   
3295                     HOMICIDIO DOLOSO  Masculino   
3296                     HOMICIDIO DOLOSO  Masculino   
3297                     HOMICIDIO DOLOSO  Masculino   
3298                     HOMICIDIO DOLOSO  Masculino   

             Escolaridade da Vítima Raça da Vítima FaixaEtaria  
0     Ensino Fundamental Incompleto  Não Informada       Idoso  
1     Ensino Fundamental Incompleto          Parda       Jovem  
2                      Alfabetizado  Não Informada       Jovem  
3       Ensino Fundamental Completo  Não Informada      Adulto  
4 

In [ ]:
df['TipoCrime'] = df['Natureza']
df['Gênero'] = df['Gênero']
df['Raça'] = df['Raça da Vítima']
df['Escolaridade'] = df['Escolaridade da Vítima']

df = df.drop('Escolaridade da Vítima', axis=1)
df = df.drop('Raça da Vítima', axis=1)
df = df.drop('Natureza', axis=1)

In [ ]:
total = len(df.index)
treino = df.iloc[: int(total * 0.7)]
teste = df.iloc[ int(total * 0.7)+1 :]

In [ ]:
def matriz_confusao(esperados, estimados):
  tp, fp, tn, fn = 0,0,0,0
  for par in zip(esperados, estimados):
    if par[0] and par[1]:
      tp += 1
    elif par[0] and not par[1]:
      fn += 1
    elif not par[0] and not par[1]:
      tn += 1
    elif not par[0] and par[1]:
      fp += 1
  return np.array([tp, fp, tn, fn])


def acuracia(esperados, estimados):
  n = len(esperados)
  tp, fp, tn, fn = matriz_confusao(esperados, estimados)
  return (tp + tn) / n



In [ ]:
colunas = [ k for k in df.keys() if k != 'Gênero']
print(colunas)

['FaixaEtaria', 'TipoCrime', 'Raça', 'Escolaridade']


In [ ]:
print(df)

         Gênero FaixaEtaria                            TipoCrime  \
0     Masculino       Idoso                     HOMICIDIO DOLOSO   
1      Feminino       Jovem      LESAO CORPORAL SEGUIDA DE MORTE   
2     Masculino       Jovem                     HOMICIDIO DOLOSO   
3     Masculino      Adulto  ROUBO SEGUIDO DE MORTE (LATROCINIO)   
4     Masculino      Adulto                     HOMICIDIO DOLOSO   
...         ...         ...                                  ...   
3294  Masculino      Adulto                     HOMICIDIO DOLOSO   
3295  Masculino      Adulto                     HOMICIDIO DOLOSO   
3296  Masculino       Idoso                     HOMICIDIO DOLOSO   
3297  Masculino      Adulto                     HOMICIDIO DOLOSO   
3298  Masculino      Adulto                     HOMICIDIO DOLOSO   

               Raça                   Escolaridade  
0     Não Informada  Ensino Fundamental Incompleto  
1             Parda  Ensino Fundamental Incompleto  
2     Não Informada     

Calculando P(x)

In [ ]:

classes = df['TipoCrime'].unique()
print(classes)

['HOMICIDIO DOLOSO' 'LESAO CORPORAL SEGUIDA DE MORTE'
 'ROUBO SEGUIDO DE MORTE (LATROCINIO)' 'FEMINICÍDIO']


In [ ]:
enconder = LabelEncoder()
sc = StandardScaler()

df['TipoCrime'] = enconder.fit_transform(df['TipoCrime'])
df['Gênero'] = enconder.fit_transform(df['Gênero'])
df['FaixaEtaria'] = enconder.fit_transform(df['FaixaEtaria'])
df['Raça'] = enconder.fit_transform(df['Raça'])
df['Escolaridade'] = enconder.fit_transform(df['Escolaridade'])

In [ ]:
enconder = LabelEncoder()
sc = StandardScaler()

#treino['TipoCrime'] = enconder.fit_transform(treino['TipoCrime'])
treino['Gênero'] = enconder.fit_transform(treino['Gênero'])
#treino['FaixaEtaria'] = enconder.fit_transform(treino['FaixaEtaria'])
#treino['Raça'] = enconder.fit_transform(treino['Raça'])
#treino['Escolaridade'] = enconder.fit_transform(treino['Escolaridade'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
def naive_bayes(df, x, y):
  n = df.size
  
  #Probabilidade da evidência - P(x)
  x_df = df.query(x)
  xn = x_df.size
  px = xn/n  

  #Probabilidade À Priori - P(y) [por classe]
  classes = df[y].unique()
  py = {}
  for classe in classes:
    y_dados = df.query("{} == {}".format(y, classe))
    yn = y_dados.size
    py[classe] = yn/n

  #Verossimilhança - P(x|y) [por classe]
  pxy = {}
  for classe in classes:
    xy_dados = df.query(x + " & {} == {}".format(y, classe))
    xyn = xy_dados.size
    pxy[classe] = (xyn/n)/py[classe]

  #Probabilidade À Posteriori - P(y|x) [por classe]
  pyx = {}
  for classe in classes:
    pyx[classe] = (pxy[classe] * py[classe]) / px

  return max(pyx, key=pyx.get)


In [ ]:
print(df)
print(treino)

      Gênero  FaixaEtaria  TipoCrime  Raça  Escolaridade
0          1            2          1     3             2
1          0            3          2     4             2
2          1            3          1     3             0
3          1            0          3     3             1
4          1            0          1     3             0
...      ...          ...        ...   ...           ...
3294       1            0          1     3             6
3295       1            0          1     3             1
3296       1            2          1     3             6
3297       1            0          1     3             3
3298       1            0          1     4             1

[3299 rows x 5 columns]
      Gênero FaixaEtaria                            TipoCrime           Raça  \
0          1       Idoso                     HOMICIDIO DOLOSO  Não Informada   
1          0       Jovem      LESAO CORPORAL SEGUIDA DE MORTE          Parda   
2          1       Jovem                     HOMICI

In [ ]:
naive_bayes(treino, "TipoCrime == 'LESAO CORPORAL SEGUIDA DE MORTE' & Raça == 'Parda'", "Gênero")

1

In [ ]:
x = ["TipoCrime","Raça"]
y = "Gênero"
esperados = []
estimados = []

for indice in teste.index:
  instancia = teste.loc[indice]

  esperados.append(instancia[y])

  tmp = ['{} == \'{}\''.format(chave, instancia[chave]) for chave in x]
  consulta = ' & '.join(tmp)

  y_est = naive_bayes(treino, consulta, y)

  estimados.append(y_est)

acuracia(esperados, estimados)


0.9908998988877654

In [ ]:
treino['TipoCrime'] = enconder.fit_transform(treino['TipoCrime'])
treino['Gênero'] = enconder.fit_transform(treino['Gênero'])
treino['FaixaEtaria'] = enconder.fit_transform(treino['FaixaEtaria'])
treino['Raça'] = enconder.fit_transform(treino['Raça'])
treino['Escolaridade'] = enconder.fit_transform(treino['Escolaridade'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
# colors = sns.color_palette("Paired")
# plt.figure(figsize=(20, 15))
# for i in range(0,len(colunas)):     
#      plt.subplot(4, 2, i+1)
#      sns.distplot(treino, x=x[i], bins=50,color=colors[i],kde=False,hist_kws=dict(alpha=1))
#      plt.xlabel(x[i],fontsize=15)

In [ ]:
acc = []
prec = []
rec = []
f1 = []
modelos = []

In [ ]:
X = df.drop('Gênero', axis=1).values
print(X)

Y= df['Gênero'].values
print(Y)

[[2 1 3 2]
 [3 2 4 2]
 [3 1 3 0]
 ...
 [2 1 3 6]
 [0 1 3 3]
 [0 1 4 1]]
[1 0 1 ... 1 1 1]


In [ ]:
## Usando a função train_test_test para
## separar os dados de testes e de treinos
X_treino,X_teste,Y_treino,Y_teste=train_test_split(X,Y,test_size=0.25,random_state=0)

In [ ]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB,CategoricalNB,MultinomialNB

naive_bernoulli = BernoulliNB()

naive_bernoulli.fit(X_treino,Y_treino)

Y_pred_bernoulli = naive_bernoulli.predict(X_teste)

acc_bernoulli = accuracy_score(Y_teste,Y_pred_bernoulli)
prec_bernoulli = precision_score(Y_teste,Y_pred_bernoulli)
rec_bernoulli = recall_score(Y_teste,Y_pred_bernoulli)
f1_bernoulli = f1_score(Y_teste,Y_pred_bernoulli)

print("Acuracia Naive Bernoulli = {:0.2f}".format(acc_bernoulli))
print("Precisão Naive Bernoulli = {:0.2f}".format(prec_bernoulli))
print("Recall Naive Bernoulli = {:0.2f}".format(rec_bernoulli))
print("F1 Naive Bernoulli = {:0.2f}".format(f1_bernoulli))

acc.append(acc_bernoulli)
prec.append(prec_bernoulli)
rec.append(rec_bernoulli)
f1.append(f1_bernoulli)
modelos.append("Naive-Bayes-Bernoulli")

Acuracia Naive Bernoulli = 0.89
Precisão Naive Bernoulli = 0.89
Recall Naive Bernoulli = 1.00
F1 Naive Bernoulli = 0.94


In [ ]:
naive_gaussian = GaussianNB()

naive_gaussian.fit(X_treino,Y_treino)

Y_pred_gaussian = naive_gaussian.predict(X_teste)

acc_gaussian = accuracy_score(Y_teste,Y_pred_gaussian)
prec_gaussian = precision_score(Y_teste,Y_pred_gaussian)
rec_gaussian = recall_score(Y_teste,Y_pred_gaussian)
f1_gaussian = f1_score(Y_teste,Y_pred_gaussian)

print("Acuracia Naive Gaussian = {:0.2f}".format(acc_gaussian))
print("Precisão Naive Gaussian = {:0.2f}".format(prec_gaussian))
print("Recall Naive Gaussian = {:0.2f}".format(rec_gaussian))
print("F1 Naive Gaussian = {:0.2f}".format(f1_gaussian))

acc.append(acc_gaussian)
prec.append(prec_gaussian)
rec.append(rec_gaussian)
f1.append(f1_gaussian)
modelos.append("Naive-Bayes_Gaussian")

Acuracia Naive Gaussian = 0.87
Precisão Naive Gaussian = 0.89
Recall Naive Gaussian = 0.98
F1 Naive Gaussian = 0.93


In [ ]:
naive_categorical = CategoricalNB()

naive_categorical.fit(X_treino,Y_treino)

Y_pred_categorical = naive_categorical.predict(X_teste)

acc_categorical = accuracy_score(Y_teste,Y_pred_categorical)
prec_categorical = precision_score(Y_teste,Y_pred_categorical)
rec_categorical = recall_score(Y_teste,Y_pred_categorical)
f1_categorical = f1_score(Y_teste,Y_pred_categorical)

print("Acuracia Naive categorical = {:0.2f}".format(acc_categorical))
print("Precisão Naive caregorical = {:0.2f}".format(prec_categorical))
print("Recall Naive categoriacal = {:0.2f}".format(rec_categorical))
print("F1 Naive categorical = {:0.2f}".format(f1_categorical))

acc.append(acc_categorical)
prec.append(prec_categorical)
rec.append(rec_categorical)
f1.append(f1_categorical)
modelos.append("Naive-Bayes-Categorical")


Acuracia Naive categorical = 0.89
Precisão Naive caregorical = 0.89
Recall Naive categoriacal = 1.00
F1 Naive categorical = 0.94


In [ ]:

naive_multi = MultinomialNB()

naive_multi.fit(X_treino,Y_treino)

Y_pred_multi = naive_multi.predict(X_teste)

acc_multi = accuracy_score(Y_teste,Y_pred_multi)
prec_multi = precision_score(Y_teste,Y_pred_multi)
rec_multi = recall_score(Y_teste,Y_pred_multi)
f1_multi = f1_score(Y_teste,Y_pred_multi)

print("Acuracia Naive Multi = {:0.2f}".format(acc_multi))
print("Precisão Naive Multi = {:0.2f}".format(prec_multi))
print("Recall Naive Multi = {:0.2f}".format(rec_multi))
print("F1 Naive Multi = {:0.2f}".format(f1_multi))

acc.append(acc_multi)
prec.append(prec_multi)
rec.append(rec_multi)
f1.append(f1_multi)
modelos.append("Naive-BayesMultinomial")


Acuracia Naive Multi = 0.88
Precisão Naive Multi = 0.88
Recall Naive Multi = 1.00
F1 Naive Multi = 0.94


In [ ]:
dici = {"Modelo" : modelos, "Acuracia" : acc, "Precisao" : prec, "Recall" : rec, "F1" : f1}

pd_dici = pd.DataFrame(dici)

pd_dici = pd_dici.sort_values(by='Recall',ascending=False)

pd_dici



,Modelo,Acuracia,Precisao,Recall,F1
0,Naive-Bayes-Bernoulli,0.887273,0.886447,1.000000,0.939806
2,Naive-Bayes-Categorical,0.887273,0.886447,1.000000,0.939806
3,Naive-BayesMultinomial,0.880000,0.880000,1.000000,0.936170
1,Naive-Bayes_Gaussian,0.870303,0.888331,0.975207,0.929744
